In [12]:
from box import ConfigBox
from ruamel.yaml import YAML
from sagemaker.pytorch.estimator import PyTorch

In [22]:
import boto3
import sagemaker
from sagemaker.estimator import Estimator

session = sagemaker.Session()

role = sagemaker.get_execution_role()

role

'arn:aws:iam::342840881361:role/service-role/AmazonSageMaker-ExecutionRole-20220507T201599'

In [8]:
%env SM_MODEL_DIR=~/example-get-started-experiments-non-demo/models
%env SM_CHANNEL_TRAIN=~/example-get-started-experiments-non-demo/data/train_data

env: SM_MODEL_DIR=~/example-get-started-experiments-non-demo/models
env: SM_CHANNEL_TRAIN=~/example-get-started-experiments-non-demo/data/train_data


In [17]:
yaml = YAML(typ="safe")
params = ConfigBox(yaml.load(open("../params.yaml", encoding="utf-8")))
args = params["base"] | params["train"]
args["fine_tune_args.epochs"] = args["fine_tune_args"]["epochs"]
args["fine_tune_args.base_lr"] = args["fine_tune_args"]["base_lr"]
del args["fine_tune_args"]
args

ConfigBox({'random_seed': 42, 'valid_pct': 0.1, 'arch': 'shufflenet_v2_x2_0', 'img_size': 256, 'batch_size': 8, 'fine_tune_args.epochs': 8, 'fine_tune_args.base_lr': 0.005})

In [23]:
pytorch_estimator = PyTorch('../src/train.py',
                            instance_type='ml.g4dn.xlarge',
                            instance_count=1,
                            framework_version='2.0.0',
                            py_version='py3',
                            hyperparameters = args,
                            role=role)
pytorch_estimator.fit({'train': 's3://example-get-started-experiments-non-demo/data/train_data'})

ValueError: Unsupported Python version: py3. You may need to upgrade your SDK version (pip install -U sagemaker) for newer Python versions. Supported Python version(s): py310.